In [63]:
# import pulp and pandas
import pulp
import pandas as pd

In [64]:
df_inputs = pd.read_csv("inputs.csv", index_col="bar")
df_inputs

,demand,w_A,w_B
bar,,,
b1,500,2,3
b2,900,4,1
b3,1800,5,3
b4,200,2,2
b5,700,1,3


In [65]:
bars = df_inputs.index.tolist()
whs = df_inputs.columns[1:].tolist()
costs = {
    wh: df_inputs[wh].to_dict()
    for wh in whs
    }
demand = df_inputs["demand"].to_dict()
capacity = {
    "w_A":1000,
    "w_B":4000
    }

In [66]:
print(bars)
print(whs)
print(costs)
print(demand)
print(capacity)

['b1', 'b2', 'b3', 'b4', 'b5']
['w_A', 'w_B']
{'w_A': {'b1': 2, 'b2': 4, 'b3': 5, 'b4': 2, 'b5': 1}, 'w_B': {'b1': 3, 'b2': 1, 'b3': 3, 'b4': 2, 'b5': 3}}
{'b1': 500, 'b2': 900, 'b3': 1800, 'b4': 200, 'b5': 700}
{'w_A': 1000, 'w_B': 4000}


In [67]:
# create model
model = pulp.LpProblem("transportation_problem", pulp.LpMinimize)

In [68]:
# create decision variables
x = pulp.LpVariable.dicts("Quantity", (whs,bars), lowBound=0)

In [69]:
# formulate objective function
model += pulp.lpSum(x[i][j] * costs[i][j] for i in whs for j in bars)

In [70]:
# add demand constraints
for j in bars:
    model += pulp.lpSum(x[i][j] for i in whs) >= demand[j], f"demand_{j}"

In [71]:
# add supply contraints
for i in whs:
    model += pulp.lpSum(x[i][j] for j in bars) <= capacity[i], f"capacity_{i}"

In [72]:
model.solve()

1

In [77]:
# Output
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Total cost: ${pulp.value(model.objective):.4f}")
for i in whs:
    for j in bars:
        print(f"{i}: {x[i][j].varValue:.2f}g")

Status: Optimal
Total cost: $8600.0000
w_A: 300.00g
w_A: 0.00g
w_A: 0.00g
w_A: 0.00g
w_A: 700.00g
w_B: 200.00g
w_B: 900.00g
w_B: 1800.00g
w_B: 200.00g
w_B: 0.00g
